# Web-scraping

In [1]:
# Librairies pour la décompression de fichiers
import gzip
from io import BytesIO

# Librairies pour le scrapping
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin

# Visualisation de données
import pandas as pd

In [28]:
# URL de la page web
url = 'https://files.data.gouv.fr/geo-dvf/latest/csv/'

response = requests.get(url)
html_content = response.content

# Utilisez BeautifulSoup pour analyser le HTML
soup = BeautifulSoup(html_content, 'html.parser')

In [29]:
# Trouver toutes les balises <a>
a_tags = soup.find_all('a')
annees=[]
# Parcourir chaque balise <a> et extraire le texte ainsi que la date/heure
for a_tag in a_tags:
    name = a_tag.text.strip()
    if name!="../":
        annees.append(name)
print(annees)

['2018/', '2019/', '2020/', '2021/', '2022/', '2023/']


In [30]:
date_element = soup.find('pre').contents[-1].strip()
date_maj = date_element.replace(" ","")
print(date_maj)

13-Oct-202320:28-


# Récupération des données sur une l'année 2023

In [31]:
# Récupération de l'avant denière année pour avoir une année comlpéte
url = f'https://files.data.gouv.fr/geo-dvf/latest/csv/{annees[-2]}'
response = requests.get(url)
html_content = response.content
soup = BeautifulSoup(html_content, 'html.parser')

# Trouver la balise 'a' qui contient les liens de téléchargement
csv_element = soup.find('pre').find('a', {'href': "full.csv.gz"})
csv_link = urljoin(url, csv_element['href'])
print(csv_link)

https://files.data.gouv.fr/geo-dvf/latest/csv/2022/full.csv.gz


In [32]:
csv_response = requests.get(csv_link)
if csv_response.status_code == 200:
    # Utilisation du buffer pour décompresser le fichier .gz
    with BytesIO(csv_response.content) as file_buffer:
        with gzip.GzipFile(fileobj=file_buffer, mode='rb') as gz_file:
            df = pd.read_csv(gz_file, low_memory=False)
            df = df.drop_duplicates()
else :
    print("Erreur !!!")

In [33]:
df.head()

,id_mutation,date_mutation,numero_disposition,nature_mutation,valeur_fonciere,adresse_numero,adresse_suffixe,adresse_nom_voie,adresse_code_voie,code_postal,...,type_local,surface_reelle_bati,nombre_pieces_principales,code_nature_culture,nature_culture,code_nature_culture_speciale,nature_culture_speciale,surface_terrain,longitude,latitude
0,2022-1,2022-01-03,1,Vente,55000.0,13.0,NaN,RUE DE LA LIBERTE,2280,1000.0,...,Appartement,24.0,1.0,NaN,NaN,NaN,NaN,NaN,5.218712,46.198053
1,2022-2,2022-01-03,1,Vente,143000.0,NaN,NaN,CHAMP COCHET,B010,1480.0,...,NaN,NaN,NaN,S,sols,NaN,NaN,84.0,4.847863,46.000587
2,2022-2,2022-01-03,1,Vente,143000.0,NaN,NaN,CHAMP COCHET,B010,1480.0,...,NaN,NaN,NaN,S,sols,NaN,NaN,88.0,4.847924,46.000581
3,2022-2,2022-01-03,1,Vente,143000.0,98.0,NaN,RTE DE LA DOMBES,0055,1480.0,...,Appartement,140.0,3.0,NaN,NaN,NaN,NaN,NaN,4.848340,46.000631
4,2022-3,2022-01-04,1,Vente,300.0,NaN,NaN,AUX PIERRES,B031,1480.0,...,NaN,NaN,NaN,T,terres,NaN,NaN,510.0,4.747594,46.045423


# EDA sur 2023

In [34]:
print(df.columns.to_list())
print(df.shape)


['id_mutation', 'date_mutation', 'numero_disposition', 'nature_mutation', 'valeur_fonciere', 'adresse_numero', 'adresse_suffixe', 'adresse_nom_voie', 'adresse_code_voie', 'code_postal', 'code_commune', 'nom_commune', 'code_departement', 'ancien_code_commune', 'ancien_nom_commune', 'id_parcelle', 'ancien_id_parcelle', 'numero_volume', 'lot1_numero', 'lot1_surface_carrez', 'lot2_numero', 'lot2_surface_carrez', 'lot3_numero', 'lot3_surface_carrez', 'lot4_numero', 'lot4_surface_carrez', 'lot5_numero', 'lot5_surface_carrez', 'nombre_lots', 'code_type_local', 'type_local', 'surface_reelle_bati', 'nombre_pieces_principales', 'code_nature_culture', 'nature_culture', 'code_nature_culture_speciale', 'nature_culture_speciale', 'surface_terrain', 'longitude', 'latitude']
(4617590, 40)


In [35]:
print("Nature de la mutation :\n",df.nature_mutation.value_counts())
print("Type de local :\n", df.type_local.value_counts())

Nature de la mutation :
 nature_mutation
Vente                                 4267222
Vente en l'état futur d'achèvement     280574
Echange                                 45200
Vente terrain à bâtir                   14268
Adjudication                             9424
Expropriation                             902
Name: count, dtype: int64
Type de local :
 type_local
Dépendance                                  1203439
Maison                                       756009
Appartement                                  638879
Local industriel. commercial ou assimilé     142535
Name: count, dtype: int64


### Selection des variables

In [36]:
df = df.loc[(df.nature_mutation=="Vente") & ((df.type_local=="Maison") | (df.type_local=="Appartement")),
                                          ["date_mutation","valeur_fonciere",
                                           "adresse_numero","adresse_nom_voie",
                                           "code_postal","code_commune","nom_commune","code_departement",
                                           "type_local","nombre_pieces_principales",
                                           "surface_reelle_bati","surface_terrain",
                                           "longitude", "latitude"]]
df.head()

,date_mutation,valeur_fonciere,adresse_numero,adresse_nom_voie,code_postal,code_commune,nom_commune,code_departement,type_local,nombre_pieces_principales,surface_reelle_bati,surface_terrain,longitude,latitude
0,2022-01-03,55000.0,13.0,RUE DE LA LIBERTE,1000.0,01053,Bourg-en-Bresse,01,Appartement,1.0,24.0,NaN,5.218712,46.198053
3,2022-01-03,143000.0,98.0,RTE DE LA DOMBES,1480.0,01398,Savigneux,01,Appartement,3.0,140.0,NaN,4.848340,46.000631
5,2022-01-06,255000.0,282.0,RTE DE POISATON,1560.0,01230,Mantenay-Montlin,01,Maison,5.0,108.0,649.0,5.103407,46.422348
8,2022-01-03,525000.0,217.0,PL DE LA CROIX BLANCHE,1390.0,01333,Saint-André-de-Corcy,01,Appartement,4.0,126.0,628.0,4.951266,45.926511
12,2022-01-05,64000.0,12.0,BD DE BROU,1000.0,01053,Bourg-en-Bresse,01,Appartement,2.0,117.0,NaN,5.229155,46.204984


In [37]:
print(df.shape)
print(df.isna().sum())

(1378948, 14)
date_mutation                     0
valeur_fonciere                3263
adresse_numero                 7105
adresse_nom_voie                 38
code_postal                      35
code_commune                      0
nom_commune                       0
code_departement                  0
type_local                        0
nombre_pieces_principales        77
surface_reelle_bati              77
surface_terrain              512854
longitude                     10963
latitude                      10963
dtype: int64


### Gestion des valeurs manquantes :

In [39]:
# Suppression des lignes avec une valeur foncière manquante ou des coordonnées manquantes
df = df.dropna(subset=['valeur_fonciere'])
df = df.dropna(subset=['longitude'])
df = df.dropna(subset=['latitude'])
df = df.dropna(subset=['adresse_nom_voie'])
df = df.dropna(subset=['code_postal'])
df = df.dropna(subset=['surface_reelle_bati'])


In [41]:
# remplacement des valeurs manquantes de adresse_numero et surface_terrain par 0.0
# car l'adresse n'a pas forcéméent de numéro et un appartement n'a pas de terrain
df['adresse_numero'] = df['adresse_numero'].fillna(0)
df['surface_terrain'] = df['surface_terrain'].fillna(0)

In [42]:
print(df.shape)
print(df.isna().sum())

(1364686, 14)
date_mutation                0
valeur_fonciere              0
adresse_numero               0
adresse_nom_voie             0
code_postal                  0
code_commune                 0
nom_commune                  0
code_departement             0
type_local                   0
nombre_pieces_principales    0
surface_reelle_bati          0
surface_terrain              0
longitude                    0
latitude                     0
dtype: int64


### Création d'une table rds test pour insérer les données

<img src="https://raw.githubusercontent.com/rastakoer/certif_app_immo/datagouv_to_rds/datagouv_to_rds/mld.PNG" alt="Schema de la bdd">


#### Création d'une table test pour valider le type de données

In [16]:
from connection import db,cursor

def create_database_and_tables():
    query="""
    CREATE DATABASE IF NOT EXISTS datagouv;
    """
    cursor.execute(query)

    query="""
    USE datagouv;
    """
    cursor.execute(query)

    # Lecture de fichier create_tables
    with open('create_tables.sql') as file:
        sql_queries = file.read()
    try :
        for query in sql_queries.split(';'):
            query = query.strip()
            print(query)
            cursor.execute(query)
    except Exception as e:
        print(f"Erreur dans la creation des tables : {e}")
        db.rollback()

    query="""
    SHOW TABLES;
    """
    cursor.execute(query)
    tables = cursor.fetchall()

    # Afficher les tables
    for table in tables:
        print(f"La table {table[0]} à bien été créée")
    return

def delete_tables():
    cursor.execute("SET foreign_key_checks = 0")
    cursor.execute("SHOW TABLES;")
    # Parcourir la liste des tables et supprimer chacune d'entre elles
    for table_name in cursor.fetchall():
        drop_table_query = f"DROP TABLE {table_name[0]}"
        cursor.execute(drop_table_query)
        print(f"Table {table_name} supprimée avec succès.")

    # Valider et appliquer les modifications
    db.commit()
    return

def closing_connection():
    cursor.close()
    db.close()
    return

In [17]:
create_database_and_tables()

#------------------------------------------------------------
#        Script MySQL.
#------------------------------------------------------------


#------------------------------------------------------------
# Table: REGIONS
#------------------------------------------------------------

CREATE TABLE IF NOT EXISTS REGIONS(
        ID_REGION   Int  Auto_increment  NOT NULL ,
        Name_region Varchar (50) NOT NULL
	,CONSTRAINT REGIONS_PK PRIMARY KEY (ID_REGION)
)ENGINE=InnoDB
#------------------------------------------------------------
# Table: DEPARTEMENTS
#------------------------------------------------------------

CREATE TABLE IF NOT EXISTS DEPARTEMENTS(
        ID_DEPT          Varchar (3) NOT NULL ,
        Name_departement Varchar (50) NOT NULL ,
        ID_REGION        Int NOT NULL
	,CONSTRAINT DEPARTEMENTS_PK PRIMARY KEY (ID_DEPT)

	,CONSTRAINT DEPARTEMENTS_REGIONS_FK FOREIGN KEY (ID_REGION) REFERENCES REGIONS(ID_REGION)
)ENGINE=InnoDB
#----------------------------------

In [18]:
delete_tables()

Table ('COMMUNES',) supprimée avec succès.
Table ('DEPARTEMENTS',) supprimée avec succès.
Table ('REGIONS',) supprimée avec succès.
Table ('TYPES_BIENS',) supprimée avec succès.
Table ('VENTES',) supprimée avec succès.


In [20]:
closing_connection()

### Récupération des noms de communes, départements et région depuis l'api data.gouv 

In [2]:
### Récupération des codes communes , noms communes et code départements 
url = "https://geo.api.gouv.fr/communes"
response = requests.get(url)
data_communes = response.json()
df_communes = pd.DataFrame(data_communes)
df_communes = df_communes.loc[:,["code","nom","codeDepartement"]]
df_communes = df_communes.drop_duplicates()
df_communes.columns= ["ID_COMMUNE","NAME_COMMUNE","ID_DEPT"]
df_communes.head()

,ID_COMMUNE,NAME_COMMUNE,ID_DEPT
0,01001,L'Abergement-Clémenciat,01
1,01002,L'Abergement-de-Varey,01
2,01004,Ambérieu-en-Bugey,01
3,01005,Ambérieux-en-Dombes,01
4,01006,Ambléon,01


In [2]:
# Récupération des codes departements, noms departements et code régions
url = "https://geo.api.gouv.fr/departements"
response = requests.get(url)
data_dpts = response.json()
df_depts = pd.DataFrame(data_dpts)
df_depts = df_depts.loc[:,["code","nom","codeRegion"]]
df_depts = df_depts.drop_duplicates()
df_depts.columns = ['ID_DEPT', 'Name_departement', 'ID_REGION']
df_depts.head()

ConnectionError: HTTPSConnectionPool(host='geo.api.gouv.fr', port=443): Max retries exceeded with url: /departements (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7ff454a77430>: Failed to establish a new connection: [Errno -3] Temporary failure in name resolution'))

In [3]:
url_regions = "https://geo.api.gouv.fr/regions"
response_regions = requests.get(url_regions)
data_regions = response_regions.json()
df_regions = pd.DataFrame(data_regions)
df_regions = df_regions.loc[:,["code","nom"]]
df_regions = df_regions.drop_duplicates()
df_regions.columns = ['ID_REGION','Name_region']
df_regions.head()

,ID_REGION,Name_region
0,11,Île-de-France
1,24,Centre-Val de Loire
2,27,Bourgogne-Franche-Comté
3,28,Normandie
4,32,Hauts-de-France


In [6]:
regions = df_regions.Name_region.unique()
for i in regions:
    print(i)

Île-de-France
Centre-Val de Loire
Bourgogne-Franche-Comté
Normandie
Hauts-de-France
Grand Est
Pays de la Loire
Bretagne
Nouvelle-Aquitaine
Occitanie
Auvergne-Rhône-Alpes
Provence-Alpes-Côte d'Azur
Corse
Guadeloupe
Martinique
Guyane
La Réunion
Mayotte


### Remplissage des tables annexes COMMUNES, DEPARTEMENTS, REGIONS et TYPES_BIENS

In [24]:
# Insertion dans la table REGIONS
query="""
USE datagouv;
"""
cursor.execute(query)

for index, row in df_regions.iterrows():
    query="""
    INSEERT IGNORE INTO REGIONS (ID_REGION,Name_region)
    VALUES (%s,%s)
    """
    cursor.execute(query,(row["ID_REGION"],row["Name_region"]))
db.commit()

query="""
SELECT * FROM REGIONS
"""
cursor.execute(query)
regions = cursor.fetchall()

# Afficher les tables
for region in regions:
    print(f"ID_region: {region[0]} -- Nom region : {region[1]}")


InterfaceError: (0, '')

In [ ]:
# Insertion dans la table DEPARTEMENTS
for index, row in df_depts.iterrows():
    query="""
    INSEERT IGNORE INTO DEPARTMENTS (ID_DEPT, Name_departement,	ID_REGION)
    VALUES (%s,%s,%s)
    """
    cursor.execute(query,(row["ID_DEPT"],row["Name_departement"],row["ID_REGION"]))
db.commit()

query="""
SELECT * FROM DEPARTMENTS
LIMIT 10
"""
cursor.execute(query)
dapartements = cursor.fetchall()

# Afficher les tables
for departement in dapartements:
    print(f"Id departement: {departement[0]} -- Nom departement : {departement[1]} -- Id region : {departement[2]}")

In [ ]:
# Insertion dans la table COMMUNES
for index, row in df_communes.iterrows():
    query="""
    INSEERT IGNORE INTO COMMUNES (ID_COMMUNE, NAME_COMMUNE,	ID_DEPT)
    VALUES (%s,%s,%s)
    """
    cursor.execute(query,(row["ID_COMMUNE"],row["NAME_COMMUNE"],row["ID_DEPT"]))
db.commit()

query="""
SELECT * FROM COMMUNES
LIMIT 10
"""
cursor.execute(query)
communes = cursor.fetchall()

# Afficher les tables
for commune in communes:
    print(f"Id commune: {commune[0]} -- Nom commune : {commune[1]} -- Id départment : {commune[2]}")

In [ ]:
# Insertion dans la table TYPES_BIENS
liste_type_local = df.type_local.unique()
for local in liste_type_local:
    query="""
    INSERT IGNORE INTO TYPES_BIENS (NAME_TYPE_BIEN)
    VALUES(%s)
    """
    cursor.execute(query,(local))
db.commit()

query="""
SELECT * FROM TYPES_BIENS
"""
cursor.execute(query)
types_biens = cursor.fetchall()

# Affichage du contenu de la table TYPES_BIENS
for i in types_biens:
    print(f"Id type bien : {i[0]} -- nom : {i[1]}")


### Insertion dans la table VENTES

In [ ]:
insert_data=df.iloc[:10,:]
insert_data.head()

#### Adaptation des données aux types de variables de la base de données rds

In [1]:
insert_data['date_mutation'] = pd.to_datetime(insert_data['date_mutation'], errors='coerce')
insert_data['valeur_fonciere'] = insert_data.valeur_fonciere.astype(int)
insert_data['adresse_numero'] = insert_data.adresse_nulero.astrype(str)
insert_data['adresse_nom_voie'] = insert_data.adresse_nom_voie.astrype(str)
insert_data['code_postal'] = insert_data.code_postal.astrype(int)
insert_data['code_commune'] = insert_data.code_commune.astrype(str)
insert_data['type_local'] = insert_data.type_local.astrype(str)
insert_data['nombre_pieces_principales'] = insert_data.nombre_pieces_principales.astrype(int)
insert_data['surface_relle_bati'] = insert_data.surface_relle_bati.astrype(int)
insert_data['surface_terrain'] = insert_data.surface_terrain.astrype(int)
insert_data['longitude'] = insert_data.longitude.astrype(float)
insert_data['latitude'] = insert_data.longitude.astrype(float)

SyntaxError: invalid syntax (75485383.py, line 4)

In [ ]:
for index,row in insert_data.iterrows:
    query="""
    INSERT INTO VENTES (DATE, MONTANT,
                      NUMERO_RUE, RUE , CODE_POSTAL,CODE_COMMUNE,
                      TYPE_BIEN, NB_PIECES ,
                      SURFACE_BATI,  SURFACE_TERRAIN
                      LONGITUDE, LATITUDE )
    VALUES (%s,%s,
            %s,%s,%s,%s,%s,
            (SELECT ID_TYPE_BIEN FROM TYPES_BIENS WHERE NAME_TYPE_BIEN=%s),%s,
            %s,%s,
            %s,%s,)
    """
    cursor.execute(query,row["date_mutation"],row['valeur_fonciere'],
                   row["adresse_numero"],row["adresse_nom_voie"],row["code_postal"],row["code_commune"],
                   row["type_local"],row["nombre_pieces_principales"],
                   row["surface_relle_bati"],row["surface_terrain"],
                   row["longitude"],row["latitude"])
db.commit()

In [ ]:
query="""
SELECT * FROM VENTES
"""

df_rds = pd.read_sql(query, db)
df_rds.head()

In [ ]:
query="""
SELECT
    V.ID_VENTE,
    V.MONTANT,
    V.NUMERO_RUE,
    V.RUE,
    V.CODE_POSTAL,
    V.LONGITUDE,
    V.LATITUDE,
    V.DATE_MUTATION,
    V.SURFACE_BATI,
    V.NB_PIECES,
    V.SURFACE_TERRAIN,
    TB.NAME_TYPE_BIEN,
    C.NAME_COMMUNE,
    D.Name_departement,
    R.Name_region
FROM
    VENTES V
    INNER JOIN TYPES_BIENS TB ON V.ID_TYPE_BIEN = TB.ID_TYPE_BIEN
    INNER JOIN COMMUNES C ON V.ID_COMMUNE = C.ID_COMMUNE
    INNER JOIN DEPARTEMENTS D ON C.ID_DEPT = D.ID_DEPT
    INNER JOIN REGIONS R ON D.ID_REGION = R.ID_REGION;
"""

df_all = pd.read_sql(query, db)
df_all.head()

In [ ]:
# Désactivation des contraintes de clé étrangère
cursor.execute('SET foreign_key_checks = 0')

# Suppression des tables
cursor.execute('DROP TABLE IF EXISTS VENTES')
cursor.execute('DROP TABLE IF EXISTS TYPES_BIENS')
cursor.execute('DROP TABLE IF EXISTS COMMUNES')
cursor.execute('DROP TABLE IF EXISTS DEPARTEMENTS')
cursor.execute('DROP TABLE IF EXISTS REGIONS')
cursor.execute('DROP DATABASE datagouv;')

In [ ]:
closing_connection()